# Politician Wikipedia Page Analysis
This notebook analyzes the English Wikipedia article coverage and quality per capita for politicians over different regions and countries.

Import necessary libraries.

In [81]:
import json, time, urllib.parse
import requests, pandas as pd
import os
from dotenv import load_dotenv

### Data
In this section we bring in politician and population datasets, call APIs for article revision IDs and quality scores, and merge the datasets together. The merged dataset is then output to a CSV file.

Import politician and population datasets from the CSV files.

In [3]:
politicians = pd.read_csv('politicians_by_country_AUG.2024.csv')

politicians.head()

,name,url,country
0,Majah Ha Adrif,https://en.wikipedia.org/wiki/Majah_Ha_Adrif,Afghanistan
1,Haroon al-Afghani,https://en.wikipedia.org/wiki/Haroon_al-Afghani,Afghanistan
2,Tayyab Agha,https://en.wikipedia.org/wiki/Tayyab_Agha,Afghanistan
3,Khadija Zahra Ahmadi,https://en.wikipedia.org/wiki/Khadija_Zahra_Ah...,Afghanistan
4,Aziza Ahmadyar,https://en.wikipedia.org/wiki/Aziza_Ahmadyar,Afghanistan


Clean the politician data by filtering out articles that aren't for individual politicians. These keywords were created by looking at the data and looking for patterns. Note that this set of keywords is not comprehensive.

In [31]:
# cleaning politician data
poli_filter = []

poli_filter = poli_filter + politicians[politicians['name'].str.contains('Council of')]['name'].tolist()

poli_filter = poli_filter + politicians[politicians['name'].str.contains('family')]['name'].tolist()

poli_filter = poli_filter + politicians[politicians['name'].str.contains('President')]['name'].tolist()

In [32]:
poli_filter

['High Council of Regency',
 'Jieng Council of Elders',
 'Kryeziu family',
 'Qatawi family',
 'Soares family',
 'Pande family',
 'Mugabe family',
 'Chairman (President) of Bulgaria',
 'President of Cuba',
 'President of the Dominican Republic',
 'Vice President of the United Arab Emirates',
 'President of Haiti',
 'President of Vanuatu']

In [33]:
politicians = politicians[~politicians['name'].isin(poli_filter)]

Assign each country to a region using the hierarchical structure of the data. We assume that regions are in all caps.

In [ ]:
population = pd.read_csv('population_by_country_AUG.2024.csv')
# move regions into its own column, with each country assigned to its lowest hierarchy region

# look for the lowest hierarchy region by checking if the next row is also in all caps
current_region = None
regions = []
for index, row in population.iterrows():
    if index + 1 < len(population):
        next_value = population.loc[index + 1, 'Geography']
    else:
        pass
    if row['Geography'].isupper() and not next_value.isupper():
        current_region = row['Geography']
        regions.append('placeholder')
    elif not row['Geography'].isupper():
        regions.append(current_region)
    else:
        regions.append('placeholder')

# add the regions to the DataFrame
population['region'] = regions

# filter out non-country rows
population = population[~population['Geography'].str.isupper()]

# reset the index
population.reset_index(drop=True, inplace=True)

population.head()

We use the [MediaWiki REST API for the EN Wikipedia](https://www.mediawiki.org/wiki/API:Main_page) to access page info data for each of the articles.

The below API code is sourced from [page info code example, Revision 1.2 - September 16, 2024](https://drive.google.com/file/d/1iGH_pOMlspeCwDzKCPRlQdq73iS16R6k/view?usp=drive_link) developed by Dr. David W. McDonald for use in DATA 512, a course in the UW MS Data Science degree program. This code is provided under the [Creative Commons](https://creativecommons.org) [CC-BY license](https://creativecommons.org/licenses/by/4.0/).
We use the API to get each page's latest revision ID. This revision ID is then used to look for the quality score.

In [34]:
# API set up - page info
#########
#
#    CONSTANTS
#

# The basic English Wikipedia API endpoint
API_ENWIKIPEDIA_ENDPOINT = "https://en.wikipedia.org/w/api.php"
API_HEADER_AGENT = 'User-Agent'

# We'll assume that there needs to be some throttling for these requests - we should always be nice to a free data resource
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (1.0/100.0)-API_LATENCY_ASSUMED

# When making automated requests we should include something that is unique to the person making the request
# This should include an email - your UW email would be good to put in there
REQUEST_HEADERS = {
    'User-Agent': '<ezhang33@uw.edu>, University of Washington, MSDS DATA 512 - AUTUMN 2024'
}

# This is just a list of English Wikipedia article titles that we can use for example requests
ARTICLE_TITLES = politicians['name'].tolist()

# This is a string of additional page properties that can be returned see the Info documentation for
# what can be included. If you don't want any this can simply be the empty string
PAGEINFO_EXTENDED_PROPERTIES = "talkid|url|watched|watchers"
#PAGEINFO_EXTENDED_PROPERTIES = ""

# This template lists the basic parameters for making this
PAGEINFO_PARAMS_TEMPLATE = {
    "action": "query",
    "format": "json",
    "titles": "",      
    "prop": "info",
    "inprop": PAGEINFO_EXTENDED_PROPERTIES
}

In [35]:
#########
#
#    PROCEDURES/FUNCTIONS
#

def request_pageinfo_per_article(article_title = None, 
                                 endpoint_url = API_ENWIKIPEDIA_ENDPOINT, 
                                 request_template = PAGEINFO_PARAMS_TEMPLATE,
                                 headers = REQUEST_HEADERS):
    
    # article title can be as a parameter to the call or in the request_template
    if article_title:
        request_template['titles'] = article_title

    if not request_template['titles']:
        raise Exception("Must supply an article title to make a pageinfo request.")

    if API_HEADER_AGENT not in headers:
        raise Exception(f"The header data should include a '{API_HEADER_AGENT}' field that contains your UW email address.")

    if 'uwnetid@uw' in headers[API_HEADER_AGENT]:
        raise Exception(f"Use your UW email address in the '{API_HEADER_AGENT}' field.")

    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free
        # data source like Wikipedia - or any other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(endpoint_url, headers=headers, params=request_template)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response


We loop through each article title and call the API function. The output is stored as a dictionary with the article titles as keys.

Missing pages return a page ID of -1. We use this to get a list of pages that do not exist.

In [72]:
revision_ids = {}
no_page = []
for i in range(len(ARTICLE_TITLES)):
    print(f"Getting page info data for: {ARTICLE_TITLES[i]}")
    info = request_pageinfo_per_article(ARTICLE_TITLES[i])
    for key, value in info['query']['pages'].items():
        if '-1' in info['query']['pages']:
            no_page.append(ARTICLE_TITLES[i])
        else:
            revision_ids[ARTICLE_TITLES[i]] = value['lastrevid'] 

Getting page info data for: Majah Ha Adrif
Getting page info data for: Haroon al-Afghani
Getting page info data for: Tayyab Agha
Getting page info data for: Khadija Zahra Ahmadi
Getting page info data for: Aziza Ahmadyar
Getting page info data for: Muqadasa Ahmadzai
Getting page info data for: Mohammad Sarwar Ahmedzai
Getting page info data for: Amir Muhammad Akhundzada
Getting page info data for: Nasrullah Baryalai Arsalai
Getting page info data for: Abdul Rahim Ayoubi
Getting page info data for: Ismael Balkhi
Getting page info data for: Abdul Baqi Turkistani
Getting page info data for: Mohammad Ghous Bashiri
Getting page info data for: Jan Baz
Getting page info data for: Bashir Ahmad Bezan
Getting page info data for: Rafiullah Bidar
Getting page info data for: Mohammad Siddiq Chakari
Getting page info data for: Cheragh Ali Cheragh
Getting page info data for: Nasir Ahmad Durrani
Getting page info data for: Muhammad Hashim Esmatullahi
Getting page info data for: Ezatullah (Nangarhar)
G

Getting page info data for: Foued Chehat
Getting page info data for: Hamza Al Sid Cheikh
Getting page info data for: Hocine Cherhabil
Getting page info data for: Abderrahmane Meziane Chérif
Getting page info data for: Saïd Chibane
Getting page info data for: Brahim Chibout
Getting page info data for: Youcef Chorfa
Getting page info data for: Fazia Dahleb
Getting page info data for: Taha Derbal
Getting page info data for: Nassim Diafat
Getting page info data for: Mokhtar Didouche
Getting page info data for: Bouras Djamel
Getting page info data for: Fatiha Serour
Getting page info data for: Ahmed Hamiani
Getting page info data for: Nadir Kassab
Getting page info data for: Brahim Djamel Kassali
Getting page info data for: Kaoutar Krikou
Getting page info data for: Sassi Lamouri
Getting page info data for: Mohamed Maghlaoui
Getting page info data for: Amar Makhloufi
Getting page info data for: Yassine Merabi
Getting page info data for: Brahim Merad
Getting page info data for: Bachir Messai

Getting page info data for: Martín Menem
Getting page info data for: Juan Mignaburu
Getting page info data for: Guillermo Montenegro
Getting page info data for: Mariano Moreno
Getting page info data for: Marcelo Muniagurria
Getting page info data for: Fernando Nadra
Getting page info data for: Rómulo Sebastián Naón
Getting page info data for: Santiago G. O'Farrell
Getting page info data for: Alfredo Olmedo
Getting page info data for: Miguel Ángel Pesce
Getting page info data for: Carlos Petroni
Getting page info data for: Alejo Peyret
Getting page info data for: Aldo Pignanelli
Getting page info data for: Abel Posse
Getting page info data for: Enrique S. Quintana
Getting page info data for: Amán Rawson
Getting page info data for: Martín Sabbatella
Getting page info data for: Juan Pablo Schiavi
Getting page info data for: Héctor Tizón
Getting page info data for: Claudio Uberti
Getting page info data for: Arístides Villanueva
Getting page info data for: Jorge Yoma
Getting page info data 

Getting page info data for: Ajdar Ismailov
Getting page info data for: Saftar Jafarov
Getting page info data for: Jeyhun Jalilov
Getting page info data for: Arastun Javadov
Getting page info data for: Rovshan Javadov
Getting page info data for: Anar Karimov
Getting page info data for: Boris Kevorkov
Getting page info data for: Gurban Khalilov
Getting page info data for: Ahmad Kordary
Getting page info data for: Lutfali bey Behbudov
Getting page info data for: Ilham Madatov
Getting page info data for: Makhmud Mamed-Guliev
Getting page info data for: Majnun Mammadov
Getting page info data for: Musa Mammadov
Getting page info data for: Ramin Mammadov
Getting page info data for: Elshad Mirbashir oghlu
Getting page info data for: Tahir Mirkishili
Getting page info data for: Movsum Samadov
Getting page info data for: Fuad Naghiyev
Getting page info data for: Eldar Namazov
Getting page info data for: Namig Nasrullayev
Getting page info data for: Siyavush Novruzov
Getting page info data for: Z

Getting page info data for: Reginald Farley
Getting page info data for: Adrian Forde
Getting page info data for: Cynthia Y. Forde
Getting page info data for: Ian Gooding-Edghill
Getting page info data for: Damien K. A. Griffith
Getting page info data for: Crystal Haynes
Getting page info data for: Philip St. Hill
Getting page info data for: Kirk Matthew Humphrey
Getting page info data for: Davidson Ishmael
Getting page info data for: Colin Everett Jordan
Getting page info data for: John Andrew King
Getting page info data for: Shantal Munro-Knight
Getting page info data for: Glyne Murray
Getting page info data for: Charles Duncan O'Neal
Getting page info data for: Samuel Jackman Prescod
Getting page info data for: Shanika Roberts-Odle
Getting page info data for: Ryan Straughn
Getting page info data for: Dwight Sutherland
Getting page info data for: Kerrie Symmonds
Getting page info data for: Yury Ambrazevich
Getting page info data for: Aliaksandr Azarau
Getting page info data for: Lyavo

...

Getting page info data for: Benjamin Giezendanner
Getting page info data for: Urs Glutz von Blotzheim
Getting page info data for: Jean-Pierre Grin-Hofmann
Getting page info data for: Lars Guggisberg
Getting page info data for: Petermann von Gundoldingen
Getting page info data for: Martin Haab
Getting page info data for: Hans-Ulrich Bigler
Getting page info data for: André Hediger
Getting page info data for: Heidi Z'graggen
Getting page info data for: Georg Friedrich Heilmann
Getting page info data for: Thomas Helbling
Getting page info data for: David Hess (painter)
Getting page info data for: Karl Otto Hunziker
Getting page info data for: Jakob Stark
Getting page info data for: Jean-Pierre Hocké
Getting page info data for: Walter Kälin
Getting page info data for: Thomas Knutti
Getting page info data for: Kris Vietze
Getting page info data for: Frédéric-César de La Harpe
Getting page info data for: Georgie Lamon
Getting page info data for: Hubert Lauper
Getting page info data for: Paul

Getting page info data for: Sang Phathanothai
Getting page info data for: Sereepisuth Temeeyaves
Getting page info data for: Singsuk Singpai
Getting page info data for: So Sethaputra
Getting page info data for: Sodsai Rungphothong
Getting page info data for: Sophon Suphapong
Getting page info data for: Suchatvee Suwansawat
Getting page info data for: Surangkana Wayuparb
Getting page info data for: Suriya Chindawongse
Getting page info data for: Sutin Klungsang
Getting page info data for: Teerakiat Jaroensettasin
Getting page info data for: Thamanat Prompow
Getting page info data for: Tiang Sirikhanth
Getting page info data for: Vichien Khaokham
Getting page info data for: Luang Wichitwathakan
Getting page info data for: Yat Hwaidi
Getting page info data for: Jacques Amouzou
Getting page info data for: Myriam Dossou D'Almeida
Getting page info data for: Issifou Tabiou Taffa
Getting page info data for: Kodjo Amegnisso Tossou
Getting page info data for: Clément Nyaletsossi Voule
Getting p

Getting page info data for: Jane Kabajungu Bainomugisha
Getting page info data for: John Kabaireho
Getting page info data for: Joseph Kabuleta
Getting page info data for: Apollo Kaggwa
Getting page info data for: Michael Kawalya Kagwa
Getting page info data for: Robert Mugabe Kakyebezi
Getting page info data for: Joselyn Kamateneti
Getting page info data for: Amos Mubunga Kambere
Getting page info data for: Paulo Kavuma
Getting page info data for: Patrick Kayemba
Getting page info data for: Kabuye Frank Kibirige
Getting page info data for: Julia Kibubura
Getting page info data for: Daniel Kidega
Getting page info data for: Yabezi Kiiza
Getting page info data for: Beatrice Kiraso
Getting page info data for: Kiwanuka Sulaiman
Getting page info data for: Ruth Molly Lematia
Getting page info data for: Annie Logiel
Getting page info data for: Ajok Lucy
Getting page info data for: Jack Odur Lutanywa
Getting page info data for: James Mamawi
Getting page info data for: Margaret Rwebyambu
Getti

Getting page info data for: Bakhodir Kurbanov (general)
Getting page info data for: Jahangir Mamatov
Getting page info data for: Rimajon Xudoyberganova
Getting page info data for: Boʻritosh Mustafoyev
Getting page info data for: Bahrom Norqobilov
Getting page info data for: Narzullo Oblomurodov
Getting page info data for: Ravshanbek Qurbonov
Getting page info data for: Baxtiyor Rahimov
Getting page info data for: Bekjon Rakhmonov
Getting page info data for: Bakhodir Khan Turkistan
Getting page info data for: Qobul Tursunov
Getting page info data for: Vincent Lunabek
Getting page info data for: Jimmy Stevens (politician)
Getting page info data for: Dinh Van Than
Getting page info data for: Fernando Adames Torres
Getting page info data for: Ivonne Attas
Getting page info data for: Fidel Barbarito
Getting page info data for: Pedro Bastidas
Getting page info data for: Alejandra Benítez
Getting page info data for: José Brito (politician)
Getting page info data for: Buenaventura Macabeo Mald

Getting page info data for: Max Mnkandla
Getting page info data for: Bekithemba Mpofu
Getting page info data for: Earnest Mudzengi
Getting page info data for: Kisinoti Mukwazhe
Getting page info data for: Edwin Mushoriwa
Getting page info data for: Munacho Mutezo
Getting page info data for: Abel Muzorewa
Getting page info data for: Prag Lalloo Naran
Getting page info data for: Owen Ncube
Getting page info data for: Nhlanhlayamangwe Felix Ndiweni
Getting page info data for: Gundwane Ndiweni
Getting page info data for: Khayisa Ndiweni
Getting page info data for: Jane Ngwenya
Getting page info data for: Maurice Nyagumbo
Getting page info data for: George Nyandoro
Getting page info data for: Sekai Nzenza
Getting page info data for: Engelbert Rugeje
Getting page info data for: Tinos Rusere
Getting page info data for: Scot Sakupwanya
Getting page info data for: Daniel Shumba
Getting page info data for: Maxwell Zeb Shumba
Getting page info data for: Nkululeko Mkastos Sibanda
Getting page info

We then use the revision IDs to call the [ORES API](https://ores.wikimedia.org/) to get article quality.

The below API code is sourced from [ORES code example, Revision 1.0 - August 15, 2023](https://drive.google.com/file/d/1GN1ULxKombHRzVsNKzj7tBhnBrSWUWXc/view?usp=drive_link) developed by Dr. David W. McDonald for use in DATA 512, a course in the UW MS Data Science degree program. This code is provided under the [Creative Commons](https://creativecommons.org) [CC-BY license](https://creativecommons.org/licenses/by/4.0/).

The API calls to the ORES API require a Wikimedia account and authentication token. Refer to [this page](https://api.wikimedia.org/wiki/Authentication) for instructions on creating the token. In the below code, we load keys from env to prevent having the token written in plain text in the notebook.

In [89]:
# ORES API
#########
#
#    CONSTANTS
#

# load keys from env

load_dotenv()

USERNAME = "Ezhang33"
ACCESS_TOKEN = os.getenv('WIKIMEDIA_ACCESS_TOKEN')


#    The current LiftWing ORES API endpoint and prediction model
#
API_ORES_LIFTWING_ENDPOINT = "https://api.wikimedia.org/service/lw/inference/v1/models/{model_name}:predict"
API_ORES_EN_QUALITY_MODEL = "enwiki-articlequality"

#
#    The throttling rate is a function of the Access token that you are granted when you request the token. The constants
#    come from dissecting the token and getting the rate limits from the granted token. An example of that is below.
#
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = ((60.0*60.0)/5000.0)-API_LATENCY_ASSUMED  # The key authorizes 5000 requests per hour

#    When making automated requests we should include something that is unique to the person making the request
#    This should include an email - your UW email would be good to put in there
#    
#    Because all LiftWing API requests require some form of authentication, you need to provide your access token
#    as part of the header too
#

email_address = 'ezhang33@uw.edu'

REQUEST_HEADER_TEMPLATE = {
    'User-Agent': f"<{email_address}>, University of Washington, MSDS DATA 512 - AUTUMN 2024",
    'Content-Type': 'application/json',
    'Authorization': f"Bearer {ACCESS_TOKEN}"
}
#
#    This is a template for the parameters that we need to supply in the headers of an API request
#
REQUEST_HEADER_PARAMS_TEMPLATE = {
    'email_address' : email_address,         # your email address should go here
    'access_token'  : ACCESS_TOKEN          # the access token you create will need to go here
}

#
#    A dictionary of English Wikipedia article titles (keys) and sample revision IDs that can be used for this ORES scoring example
#
ARTICLE_REVISIONS = revision_ids

#
#    This is a template of the data required as a payload when making a scoring request of the ORES model
#
ORES_REQUEST_DATA_TEMPLATE = {
    "lang":        "en",     # required that its english - we're scoring English Wikipedia revisions
    "rev_id":      "",       # this request requires a revision id
    "features":    True
}

#
#    These are used later - defined here so they, at least, have empty values
#
USERNAME = ""
ACCESS_TOKEN = ""
#

In [90]:
# ORES API function

#########
#
#    PROCEDURES/FUNCTIONS
#

def request_ores_score_per_article(article_revid = None, email_address=None, access_token=None,
                                   endpoint_url = API_ORES_LIFTWING_ENDPOINT, 
                                   model_name = API_ORES_EN_QUALITY_MODEL, 
                                   request_data = ORES_REQUEST_DATA_TEMPLATE, 
                                   header_format = REQUEST_HEADER_TEMPLATE, 
                                   header_params = REQUEST_HEADER_PARAMS_TEMPLATE):
    
    #    Make sure we have an article revision id, email and token
    #    This approach prioritizes the parameters passed in when making the call
    if article_revid:
        request_data['rev_id'] = article_revid
    if email_address:
        header_params['email_address'] = email_address
    if access_token:
        header_params['access_token'] = access_token
    
    #   Making a request requires a revision id - an email address - and the access token
    if not request_data['rev_id']:
        raise Exception("Must provide an article revision id (rev_id) to score articles")
    if not header_params['email_address']:
        raise Exception("Must provide an 'email_address' value")
    if not header_params['access_token']:
        raise Exception("Must provide an 'access_token' value")
    
    # Create the request URL with the specified model parameter - default is a article quality score request
    request_url = endpoint_url.format(model_name=model_name)
    
    # Create a compliant request header from the template and the supplied parameters
    headers = dict()
    for key in header_format.keys():
        headers[str(key)] = header_format[key].format(**header_params)
    
    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free data
        # source like ORES - or other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        #response = requests.get(request_url, headers=headers)
        response = requests.post(request_url, headers=headers, data=json.dumps(request_data))
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response

We loop through each of the revision IDs to get the quality score for each article and store in a dictionary.

In [102]:
ores_scores = {}
no_score = []
for article_title in ARTICLE_REVISIONS:
    print(f"Getting LiftWing ORES scores for '{article_title}' with revid: {ARTICLE_REVISIONS[article_title]:d}")
    
    score = request_ores_score_per_article(article_revid=ARTICLE_REVISIONS[article_title],
                                       email_address=email_address,
                                       access_token=ACCESS_TOKEN)
    prediction = score['enwiki']['scores'][f'{ARTICLE_REVISIONS[article_title]:d}']['articlequality']['score']['prediction']
    ores_scores[article_title] = prediction

Getting LiftWing ORES scores for 'Majah Ha Adrif' with revid: 1233202991
Getting LiftWing ORES scores for 'Haroon al-Afghani' with revid: 1230459615
Getting LiftWing ORES scores for 'Tayyab Agha' with revid: 1225661708
Getting LiftWing ORES scores for 'Khadija Zahra Ahmadi' with revid: 1234741562
Getting LiftWing ORES scores for 'Aziza Ahmadyar' with revid: 1195651393
Getting LiftWing ORES scores for 'Muqadasa Ahmadzai' with revid: 1235521766
Getting LiftWing ORES scores for 'Mohammad Sarwar Ahmedzai' with revid: 1176429234
Getting LiftWing ORES scores for 'Amir Muhammad Akhundzada' with revid: 1247931713
Getting LiftWing ORES scores for 'Nasrullah Baryalai Arsalai' with revid: 1225385278
Getting LiftWing ORES scores for 'Abdul Rahim Ayoubi' with revid: 1226326055
Getting LiftWing ORES scores for 'Ismael Balkhi' with revid: 1244521219
Getting LiftWing ORES scores for 'Abdul Baqi Turkistani' with revid: 1231655023
Getting LiftWing ORES scores for 'Mohammad Ghous Bashiri' with revid: 123

Getting LiftWing ORES scores for 'Qamil Musa Haxhi Feza' with revid: 1198224146
Getting LiftWing ORES scores for 'Themistokli Gërmenji' with revid: 1241037317
Getting LiftWing ORES scores for 'Sabri Godo' with revid: 1230739582
Getting LiftWing ORES scores for 'Mihal Grameno' with revid: 1226635783
Getting LiftWing ORES scores for 'Aćif Hadžiahmetović' with revid: 1229938943
Getting LiftWing ORES scores for 'Kurt Agë Kadiu' with revid: 1244486529
Getting LiftWing ORES scores for 'Mehmed Kalakula' with revid: 1145510350
Getting LiftWing ORES scores for 'Alfred Karamuço' with revid: 1098236555
Getting LiftWing ORES scores for 'Ibrahim Karbunara' with revid: 1235824719
Getting LiftWing ORES scores for 'Ali Këlcyra' with revid: 1227369252
Getting LiftWing ORES scores for 'Bajram Kelmendi' with revid: 1237006808
Getting LiftWing ORES scores for 'Kristo Kirka' with revid: 1237950375
Getting LiftWing ORES scores for 'Elez Koçi' with revid: 1174010569
Getting LiftWing ORES scores for 'Xhelal K

Getting LiftWing ORES scores for 'Abderrachid Tabbi' with revid: 1218530483
Getting LiftWing ORES scores for 'Karim Bibi Triki' with revid: 1213794210
Getting LiftWing ORES scores for 'Layachi Yaker' with revid: 1226060720
Getting LiftWing ORES scores for 'Ahmed Zeghdar' with revid: 1213778323
Getting LiftWing ORES scores for 'Adão Francisco Correia de Almeida' with revid: 1209524394
Getting LiftWing ORES scores for 'Ana Paula Chantre Luna de Carvalho' with revid: 1209514022
Getting LiftWing ORES scores for 'Alexandre Sebastião André' with revid: 1172178750
Getting LiftWing ORES scores for 'Paulino Baptista' with revid: 1242307496
Getting LiftWing ORES scores for 'Bernarda Gonçalves Martins' with revid: 1210193481
Getting LiftWing ORES scores for 'João Baptista Borges' with revid: 1227794006
Getting LiftWing ORES scores for 'Carolina Cerqueira' with revid: 1241977204
Getting LiftWing ORES scores for 'Arlete Chimbinda' with revid: 1177486404
Getting LiftWing ORES scores for 'Kafundanga 

Getting LiftWing ORES scores for 'Silvano Bores' with revid: 1225775647
Getting LiftWing ORES scores for 'Exequiel Bustillo' with revid: 1188044851
Getting LiftWing ORES scores for 'Mario Cafiero' with revid: 1135193654
Getting LiftWing ORES scores for 'Alberto Manuel Campos' with revid: 1166331326
Getting LiftWing ORES scores for 'Alejo Castex' with revid: 1231276700
Getting LiftWing ORES scores for 'Dámaso Centeno' with revid: 1245130520
Getting LiftWing ORES scores for 'Rossana Chahla' with revid: 1227517224
Getting LiftWing ORES scores for 'Lázaro Chiappe' with revid: 1232135530
Getting LiftWing ORES scores for 'Carlos Corach' with revid: 1227620960
Getting LiftWing ORES scores for 'Jorge Coscia' with revid: 1227459720
Getting LiftWing ORES scores for 'Carlos de la Rosa' with revid: 1209668330
Getting LiftWing ORES scores for 'Juan Carlos Del Bello' with revid: 1225784914
Getting LiftWing ORES scores for 'Luis L. Domínguez' with revid: 1234469779
Getting LiftWing ORES scores for 'M

Getting LiftWing ORES scores for 'Ernst Hampel (politician)' with revid: 1232067132
Getting LiftWing ORES scores for 'Hans Peter Haselsteiner' with revid: 1226351964
Getting LiftWing ORES scores for 'Friedrich Wilhelm von Haugwitz' with revid: 1215043658
Getting LiftWing ORES scores for 'Eduard Hedvicek' with revid: 1215043825
Getting LiftWing ORES scores for 'Hermann Stern' with revid: 1215044008
Getting LiftWing ORES scores for 'Jürgen Himmelbauer' with revid: 1215045853
Getting LiftWing ORES scores for 'Anton Hofer' with revid: 1215044432
Getting LiftWing ORES scores for 'Andreas Hörtnagl' with revid: 1215046038
Getting LiftWing ORES scores for 'Franz Jetzinger' with revid: 1214236945
Getting LiftWing ORES scores for 'Konstantin Jireček' with revid: 1242771327
Getting LiftWing ORES scores for 'Hugo Jury' with revid: 1225140706
Getting LiftWing ORES scores for 'Joseph Kenner' with revid: 1169137921
Getting LiftWing ORES scores for 'Theodor Kery' with revid: 1215046740
Getting LiftWin

...

Getting LiftWing ORES scores for 'Bekbol Sagyn' with revid: 1165541969
Getting LiftWing ORES scores for 'Marat Sarsembaev' with revid: 1221862976
Getting LiftWing ORES scores for 'Jambulat Sarsenov' with revid: 1250259527
Getting LiftWing ORES scores for 'Amangeldy Shabdarbayev' with revid: 1243509989
Getting LiftWing ORES scores for 'Adil Shayakhmetov' with revid: 1243509994
Getting LiftWing ORES scores for 'Zhanat Suleimenov' with revid: 1148774732
Getting LiftWing ORES scores for 'Serik Sultangali' with revid: 1243509672
Getting LiftWing ORES scores for 'Abai Tasbolatov' with revid: 1243509983
Getting LiftWing ORES scores for 'Kuanyshbek Yessekeev' with revid: 1245284558
Getting LiftWing ORES scores for 'Zhagda Babalykov' with revid: 1210320561
Getting LiftWing ORES scores for 'Ahmed Abass' with revid: 1246061443
Getting LiftWing ORES scores for 'Abdilatif Abdalla' with revid: 1241910337
Getting LiftWing ORES scores for 'Mohamed Huka Adan' with revid: 1246061448
Getting LiftWing ORE

Getting LiftWing ORES scores for 'Gloria Orwoba' with revid: 1245954425
Getting LiftWing ORES scores for 'John Owino' with revid: 1246061278
Getting LiftWing ORES scores for 'Gonzi Rai' with revid: 1246062448
Getting LiftWing ORES scores for 'Ali Roba' with revid: 1246310480
Getting LiftWing ORES scores for 'Wesley K. Rono' with revid: 1177252018
Getting LiftWing ORES scores for 'Ruweida Obo' with revid: 1147726394
Getting LiftWing ORES scores for 'Ibrahim Sane' with revid: 1246062464
Getting LiftWing ORES scores for 'Ibrahim A. Saney' with revid: 1231599424
Getting LiftWing ORES scores for 'Kimaiyo Sego' with revid: 1095169588
Getting LiftWing ORES scores for 'Cornelly Serem' with revid: 1246061286
Getting LiftWing ORES scores for 'Peter Safari Shehe' with revid: 1246062468
Getting LiftWing ORES scores for 'Mohamed Shidiye' with revid: 1246062471
Getting LiftWing ORES scores for 'Elias Shill' with revid: 1246062483
Getting LiftWing ORES scores for 'Lawrence Sifuna' with revid: 1246062

Getting LiftWing ORES scores for 'Ali Mohammed Thunayan Al-Ghanim' with revid: 1228803753
Getting LiftWing ORES scores for 'Hamad Al-Midlij' with revid: 1245823231
Getting LiftWing ORES scores for 'Ibrahim Al-Mudhaf' with revid: 1160127010
Getting LiftWing ORES scores for 'Abdullah Al-Nibari' with revid: 1096143240
Getting LiftWing ORES scores for 'Obaid Al-Wasmi' with revid: 1226748080
Getting LiftWing ORES scores for 'Salem Al-Ali Al-Sabah' with revid: 1241175409
Getting LiftWing ORES scores for 'Thamer Ali Sabah Al-Salem Al-Sabah' with revid: 1210778715
Getting LiftWing ORES scores for 'Mubarak Al-Zwair' with revid: 1223430952
Getting LiftWing ORES scores for 'Arstanbek Abdyldayev' with revid: 1220308001
Getting LiftWing ORES scores for 'Akbaraly Aitikeev' with revid: 1015639087
Getting LiftWing ORES scores for 'Bermet Akayeva' with revid: 1158549938
Getting LiftWing ORES scores for 'Maxim Bakiyev' with revid: 1223543314
Getting LiftWing ORES scores for 'Janish Bakiyev' with revid: 

Getting LiftWing ORES scores for 'Clarence Lorenzo Simpson Jr.' with revid: 1181575494
Getting LiftWing ORES scores for 'Ellen Johnson Sirleaf' with revid: 1244504845
Getting LiftWing ORES scores for 'Jeremiah Sulunteh' with revid: 1249835446
Getting LiftWing ORES scores for 'Alaric Tokpa' with revid: 1183708441
Getting LiftWing ORES scores for 'E. Reginald Townsend' with revid: 1247535227
Getting LiftWing ORES scores for 'William Tubman' with revid: 1249700084
Getting LiftWing ORES scores for 'Samuel Tweah' with revid: 1238850652
Getting LiftWing ORES scores for 'Blahsue Vonleh' with revid: 1158703245
Getting LiftWing ORES scores for 'Tom Woewiyu' with revid: 1218952304
Getting LiftWing ORES scores for 'Ahmed Rafiq Almhadoui' with revid: 1236481840
Getting LiftWing ORES scores for 'Mohamed Abulasʽad El-ʽAlem' with revid: 978820765
Getting LiftWing ORES scores for 'Abd-Rabbo al-Barassi' with revid: 1245381256
Getting LiftWing ORES scores for 'Fathi Bashagha' with revid: 1245556589
Gett

KeyError: 'enwiki'

We found that the for loop will sometimes get stuck on articles that have data. The below code is used to continue the API calls from where it left off before without overwriting existing data.

In [133]:
# continuing from 'Jean Ersfeld'...
# and again from 'Romulus Cioflec'
# and again from 'Bolesław Fleszar'
# ... 'Chato Galante'
# ... 'Arantza Urkaregi'
# ... 'Sam Peter Christopher Fernando'
# ... 'José Pedro Varela'
article_revisions_cont = {k: v for k, v in ARTICLE_REVISIONS.items() if k not in ores_scores}
for article_title in article_revisions_cont:
    print(f"Getting LiftWing ORES scores for '{article_title}' with revid: {ARTICLE_REVISIONS[article_title]:d}")
    
    score = request_ores_score_per_article(article_revid=ARTICLE_REVISIONS[article_title],
                                       email_address=email_address,
                                       access_token=ACCESS_TOKEN)
    prediction = score['enwiki']['scores'][f'{ARTICLE_REVISIONS[article_title]:d}']['articlequality']['score']['prediction']
    ores_scores[article_title] = prediction

Getting LiftWing ORES scores for 'José Pedro Varela' with revid: 1184349404
Getting LiftWing ORES scores for 'Pedro Vaz (diplomat)' with revid: 1108530199
Getting LiftWing ORES scores for 'Mateo Vidal' with revid: 1093592713
Getting LiftWing ORES scores for 'Jorge Zabalza' with revid: 1232110666
Getting LiftWing ORES scores for 'Ikhtiyor Abdullayev' with revid: 1247702597
Getting LiftWing ORES scores for 'Adham Ahmedbaev' with revid: 1247704434
Getting LiftWing ORES scores for 'Jakhongir Artikkhodjayev' with revid: 1230521779
Getting LiftWing ORES scores for 'Valery Ataev' with revid: 1205273470
Getting LiftWing ORES scores for 'Abdusalom Azizov' with revid: 1246468761
Getting LiftWing ORES scores for 'Uktam Barnoev' with revid: 1224037855
Getting LiftWing ORES scores for 'Sa'dulla Begaliyev' with revid: 1247704310
Getting LiftWing ORES scores for 'Rustam Inoyatov' with revid: 1247702441
Getting LiftWing ORES scores for 'Rashid Kadyrov' with revid: 1247702823
Getting LiftWing ORES scor

Getting LiftWing ORES scores for 'Nguyễn Phan Long' with revid: 1221992596
Getting LiftWing ORES scores for 'Nguyễn Quốc Cường' with revid: 1130925190
Getting LiftWing ORES scores for 'Nguyễn Thị Hồng (economist)' with revid: 1214023850
Getting LiftWing ORES scores for 'Nguyễn Thị Thập' with revid: 1145703738
Getting LiftWing ORES scores for 'Nguyễn Văn Huyền' with revid: 1241609893
Getting LiftWing ORES scores for 'Nguyễn Văn Lực' with revid: 1106652276
Getting LiftWing ORES scores for 'Nguyễn Văn Trân' with revid: 1245671291
Getting LiftWing ORES scores for 'Nguyễn Văn Xuân' with revid: 1231154121
Getting LiftWing ORES scores for 'Phan Anh' with revid: 1224595790
Getting LiftWing ORES scores for 'Tạ Quang Bửu' with revid: 1224595798
Getting LiftWing ORES scores for 'Tran Minh Tiet' with revid: 1225066058
Getting LiftWing ORES scores for 'Trần Quốc Vượng (politician)' with revid: 1237490552
Getting LiftWing ORES scores for 'Trần Văn Hữu' with revid: 1244544286
Getting LiftWing ORES sc

Getting LiftWing ORES scores for 'Leopold Takawira' with revid: 1236900271
Getting LiftWing ORES scores for 'Gift Tandare' with revid: 1235705451
Getting LiftWing ORES scores for 'Rekayi Tangwena' with revid: 1236117496
Getting LiftWing ORES scores for 'Temba Mliswa' with revid: 1246241769
Getting LiftWing ORES scores for 'Josiah Tongogara' with revid: 1203429435
Getting LiftWing ORES scores for 'Langton Towungana' with revid: 1246280093
Getting LiftWing ORES scores for 'Sengezo Tshabangu' with revid: 1228478288
Getting LiftWing ORES scores for 'Herbert Ushewokunze' with revid: 959111842
Getting LiftWing ORES scores for 'Denis Walker' with revid: 1247902630


Here we check the error rate, defined as the number of articles without a score divided by the total number of articles.
We also check that all values in the scores dictionary are valid.

In [282]:
# error rate
# check that all values in ores_scores dictionary are valid
print(set(ores_scores.values()))

# check for null values
null_keys = [key for key, value in ores_scores.items() if value is None]
print(null_keys)

# log missing articles
missing_articles = list(set(ARTICLE_TITLES) - set(list(ores_scores.keys())))
print(f'Articles without ORES scores: {missing_articles}')

# calculate error rate (no score/total number of articles)
print(len(missing_articles)/len(ores_scores))

{'B', 'GA', 'FA', 'Stub', 'C', 'Start'}
[]
Articles without ORES scores: ['André Ngongang Ouandji', 'Kyaw Myint', 'Bashir Bililiqo', 'Tomás Pimentel', 'Richard Sumah', 'Barbara Eibinger-Miedl', "Segun ''Aeroland'' Adewale", 'Mehrali Gasimov']
0.0011283497884344146


7090

We will now merge the population, politician, revision ID, and score data. This data is then exported to a csv file.

Missing countries are exported to a txt file.

In [199]:
# merging data
# merge the politician csv with revision id and quality

# transform revision ids json into dataframe
revision_ids_df = pd.DataFrame(list(revision_ids.items()), columns = ['name', 'revision_id'])

#transform ores scores json into dataframe
ores_scores_df = pd.DataFrame(list(ores_scores.items()), columns = ['name', 'article_quality'])

# join with politician csv
politicians_revid = pd.merge(politicians, revision_ids_df, on = 'name', how = 'left')
politicians_revid_scores = pd.merge(politicians_revid, ores_scores_df, on = 'name', how = 'left')

# join with country population csv
wp_politicians_by_country = pd.merge(politicians_revid_scores, population, left_on = 'country', right_on = 'Geography', how = 'inner').drop(columns=['Geography'])

wp_politicians_by_country = wp_politicians_by_country.rename(columns = {'Population': 'population', 'name': 'article_title'})
wp_politicians_by_country = wp_politicians_by_country[['country', 'region', 'population', 'article_title', 'revision_id', 'article_quality']]

wp_politicians_by_country.head()

,country,region,population,article_title,revision_id,article_quality
0,Afghanistan,SOUTH ASIA,42.4,Majah Ha Adrif,1.233203e+09,Start
1,Afghanistan,SOUTH ASIA,42.4,Haroon al-Afghani,1.230460e+09,B
2,Afghanistan,SOUTH ASIA,42.4,Tayyab Agha,1.225662e+09,Start
3,Afghanistan,SOUTH ASIA,42.4,Khadija Zahra Ahmadi,1.234742e+09,Stub
4,Afghanistan,SOUTH ASIA,42.4,Aziza Ahmadyar,1.195651e+09,Start


In [216]:
# export to csv file
wp_politicians_by_country.to_csv('wp_politicians_by_country.csv', index=False)

In [215]:
# look for missing countries
# countries from population csv not in the final dataframe
missing_countries = population[~population['Geography'].isin(set(wp_politicians_by_country['country']))]
missing_countries = missing_countries['Geography'].tolist()

# append countries from politicians csv not in population csv
missing_countries.extend(set(politicians[~politicians['country'].isin(set(wp_politicians_by_country['country']))]['country']))
missing_countries

# write out to txt file
with open('wp_countries-no_match.txt', 'w') as file:
    for item in missing_countries:
        file.write(f"{item}\n")

['Western Sahara',
 'GuineaBissau',
 'Mauritius',
 'Mayotte',
 'Reunion',
 'Sao Tome and Principe',
 'eSwatini',
 'Canada',
 'United States',
 'Mexico',
 'Curacao',
 'Dominica',
 'Guadeloupe',
 'Jamaica',
 'Martinique',
 'Puerto Rico',
 'French Guiana',
 'Suriname',
 'Georgia',
 'Brunei',
 'Philippines',
 'China (Hong Kong SAR)',
 'China (Macao SAR)',
 'Korea (North)',
 'Korea (South)',
 'Denmark',
 'Iceland',
 'Ireland',
 'United Kingdom',
 'Liechtenstein',
 'Netherlands',
 'Romania',
 'Andorra',
 'San Marino',
 'Australia',
 'Fiji',
 'French Polynesia',
 'Guam',
 'Kiribati',
 'Nauru',
 'New Caledonia',
 'New Zealand',
 'Palau',
 'Guinea-Bissau',
 'Korean',
 'Korea, South']

### Analysis
In this section, we look at the article coverage per capita and article quality per capita. Note that the populations are represnted in millions. 

In [296]:
# create grouped dataframe
good_quality = ['FA', 'GA']
article_counts = wp_politicians_by_country.groupby('country')['article_quality'].count()
article_counts = article_counts.reset_index().rename(columns = {'article_quality': 'article_count'})

quality_counts = wp_politicians_by_country[wp_politicians_by_country['article_quality'].isin(good_quality)].groupby('country').size()
quality_counts = quality_counts.reset_index().rename(columns = {0: 'quality_count'})

counts = pd.merge(article_counts, quality_counts, on = 'country', how = 'left')
counts['quality_count'] = counts['quality_count'].fillna(0)

country_counts = pd.merge(counts, population, left_on = 'country', right_on = 'Geography', how = 'inner')
country_counts = country_counts.rename(columns = {'Population': 'population'})
country_counts

,country,article_count,quality_count,Geography,population,region
0,Afghanistan,85,3.0,Afghanistan,42.4,SOUTH ASIA
1,Albania,68,7.0,Albania,2.7,SOUTHERN EUROPE
2,Algeria,71,1.0,Algeria,46.8,NORTHERN AFRICA
3,Angola,58,2.0,Angola,36.7,MIDDLE AFRICA
4,Antigua and Barbuda,33,0.0,Antigua and Barbuda,0.1,CARIBBEAN
...,...,...,...,...,...,...
161,Venezuela,56,1.0,Venezuela,28.8,SOUTH AMERICA
162,Vietnam,36,2.0,Vietnam,98.9,SOUTHEAST ASIA
163,Yemen,32,0.0,Yemen,34.4,WESTERN ASIA
164,Zambia,3,0.0,Zambia,20.2,EASTERN AFRICA


In [298]:
# calculate coverage per capita by country
country_coverage = country_counts.copy()
country_coverage['coverage_ratio'] = country_counts['article_count']/country_counts['population']

# calculate quality per capita by country
country_quality = country_counts.copy()
country_quality['quality_ratio'] = country_counts['quality_count']/country_counts['population']

# group dataframe by region to get population totals
region_population = country_counts.groupby('region')['population'].sum()
region_population.reset_index()

# calculate coverage per capita by region
region_coverage = country_counts.groupby('region')['article_count'].sum()
region_coverage.reset_index()
region_coverage = pd.merge(region_coverage, region_population, on = 'region', how = 'inner')
region_coverage['coverage_ratio'] = region_coverage['article_count']/region_coverage['population']

# calculate quality per capita by region
region_quality = country_counts.groupby('region')['quality_count'].sum()
region_quality.reset_index()
region_quality = pd.merge(region_quality, region_population, on = 'region', how = 'inner')
region_quality['quality_ratio'] = region_quality['quality_count']/region_quality['population']

Top 10 countries by coverage, descending order. Note that there are ties between countries in the top 10, therefore outputting more than 10 countries.

In [304]:
# Top 10 countries by coverage (desc)
top_10_coverage_c = country_coverage.sort_values(by=['coverage_ratio'], ascending = False, na_position='first')[0:14]
top_10_coverage_c

,country,article_count,quality_count,Geography,population,region,coverage_ratio
154,Tuvalu,1,0.0,Tuvalu,0.0,OCEANIA,inf
96,Monaco,10,0.0,Monaco,0.0,WESTERN EUROPE,inf
4,Antigua and Barbuda,33,0.0,Antigua and Barbuda,0.1,CARIBBEAN,330.000000
51,Federated States of Micronesia,14,0.0,Federated States of Micronesia,0.1,OCEANIA,140.000000
93,Marshall Islands,13,0.0,Marshall Islands,0.1,OCEANIA,130.000000
149,Tonga,10,0.0,Tonga,0.1,OCEANIA,100.000000
12,Barbados,25,0.0,Barbados,0.3,CARIBBEAN,83.333333
125,Seychelles,6,0.0,Seychelles,0.1,EASTERN AFRICA,60.000000
98,Montenegro,36,3.0,Montenegro,0.6,SOUTHERN EUROPE,60.000000
17,Bhutan,44,0.0,Bhutan,0.8,SOUTH ASIA,55.000000


Bottom 10 countries by coverage, ascending.

In [301]:
# Bottom 10 countries by coverage (asc)
bot_10_coverage_c = country_coverage.sort_values(by=['coverage_ratio'], na_position='last')[0:10]
bot_10_coverage_c

,country,article_count,quality_count,Geography,population,region,coverage_ratio
31,China,16,0.0,China,1411.3,EAST ASIA,0.011337
57,Ghana,3,1.0,Ghana,34.1,WESTERN AFRICA,0.087977
66,India,151,0.0,India,1428.6,SOUTH ASIA,0.105698
122,Saudi Arabia,5,2.0,Saudi Arabia,36.9,WESTERN ASIA,0.135501
164,Zambia,3,0.0,Zambia,20.2,EASTERN AFRICA,0.148515
108,Norway,1,0.0,Norway,5.5,NORTHERN EUROPE,0.181818
70,Israel,2,0.0,Israel,9.8,WESTERN ASIA,0.204082
45,Egypt,30,1.0,Egypt,105.2,NORTHERN AFRICA,0.285171
37,Cote d'Ivoire,10,0.0,Cote d'Ivoire,30.9,WESTERN AFRICA,0.323625
50,Ethiopia,44,2.0,Ethiopia,126.5,EASTERN AFRICA,0.347826


Top 10 countries by high quality, descending. We define high quality as FA and GA ratings.

In [275]:
# Top 10 countries by high quality (desc)
top_10_quality_c = country_quality.sort_values(by=['quality_ratio'], ascending = False, na_position='last')[0:10]
top_10_quality_c

,country,article_count,quality_count,Geography,population,region,coverage_ratio,quality_ratio
98,Montenegro,36,3.0,Montenegro,0.6,SOUTHERN EUROPE,60.000000,5.000000
86,Luxembourg,27,2.0,Luxembourg,0.7,WESTERN EUROPE,38.571429,2.857143
1,Albania,68,7.0,Albania,2.7,SOUTHERN EUROPE,25.185185,2.592593
76,Kosovo,26,4.0,Kosovo,1.7,SOUTHERN EUROPE,15.294118,2.352941
90,Maldives,33,1.0,Maldives,0.6,SOUTH ASIA,55.000000,1.666667
85,Lithuania,58,4.0,Lithuania,2.9,NORTHERN EUROPE,20.000000,1.379310
38,Croatia,65,5.0,Croatia,3.8,SOUTHERN EUROPE,17.105263,1.315789
62,Guyana,17,1.0,Guyana,0.8,SOUTH AMERICA,21.250000,1.250000
111,Palestinian Territory,61,6.0,Palestinian Territory,5.5,WESTERN ASIA,11.090909,1.090909
129,Slovenia,38,2.0,Slovenia,2.1,SOUTHERN EUROPE,18.095238,0.952381


Bottom 10 countries by high quality, ascending.

In [277]:
# Bottom 10 countries by high quality (asc)
bot_10_quality_c = country_quality.sort_values(by=['quality_ratio'], na_position='last')[0:10]
bot_10_quality_c

,country,article_count,quality_count,Geography,population,region,coverage_ratio,quality_ratio
165,Zimbabwe,68,0.0,Zimbabwe,16.7,EASTERN AFRICA,4.071856,0.0
34,Congo,31,0.0,Congo,6.1,MIDDLE AFRICA,5.081967,0.0
77,Kuwait,17,0.0,Kuwait,4.4,WESTERN ASIA,3.863636,0.0
137,St. Lucia,3,0.0,St. Lucia,0.2,CARIBBEAN,15.000000,0.0
37,Cote d'Ivoire,10,0.0,Cote d'Ivoire,30.9,WESTERN AFRICA,0.323625,0.0
136,St. Kitts and Nevis,3,0.0,St. Kitts and Nevis,0.1,CARIBBEAN,30.000000,0.0
130,Solomon Islands,12,0.0,Solomon Islands,0.8,OCEANIA,15.000000,0.0
40,Cyprus,16,0.0,Cyprus,1.3,WESTERN ASIA,12.307692,0.0
127,Singapore,4,0.0,Singapore,5.8,SOUTHEAST ASIA,0.689655,0.0
42,Djibouti,15,0.0,Djibouti,1.1,EASTERN AFRICA,13.636364,0.0


Geographic regions by total coverage, descending. Note that North America is not in the list.

In [278]:
# Geographic regions by total coverage (desc)
region_coverage.sort_values(by=['coverage_ratio'], ascending = False, na_position='last')

,article_count,population,coverage_ratio
region,,,
NORTHERN EUROPE,191,27.8,6.870504
OCEANIA,71,11.1,6.396396
CARIBBEAN,215,36.6,5.874317
SOUTHERN EUROPE,795,151.5,5.247525
CENTRAL AMERICA,188,51.3,3.664717
WESTERN EUROPE,497,181.3,2.741313
EASTERN EUROPE,708,266.2,2.659654
WESTERN ASIA,608,295.4,2.058226
SOUTHERN AFRICA,123,68.3,1.800878


Geographic regions by high quality coverage, descending. Once again, North America is not in the list.

In [279]:
# Geographic regions by high quality coverage (desc)
region_quality.sort_values(by=['quality_ratio'], ascending = False, na_position='last')

,quality_count,population,quality_ratio
region,,,
SOUTHERN EUROPE,53.0,151.5,0.349835
NORTHERN EUROPE,9.0,27.8,0.323741
CARIBBEAN,9.0,36.6,0.245902
CENTRAL AMERICA,10.0,51.3,0.194932
EASTERN EUROPE,38.0,266.2,0.142750
SOUTHERN AFRICA,8.0,68.3,0.117130
WESTERN EUROPE,21.0,181.3,0.115830
WESTERN ASIA,27.0,295.4,0.091401
OCEANIA,1.0,11.1,0.090090
